In [1]:
import import_ipynb
import string
import re

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer

from DatasetEDA import retrieve_reviews_df, preprocessing, remove_stopwords

importing Jupyter notebook from DatasetEDA.ipynb


ModuleNotFoundError: No module named 'wordcloud'

In [5]:
def preprocess():
    # Create one dataframe with data, then drop columns and change score to 1/0/-1
    df = preprocessing(retrieve_reviews_df())

    # Save df to csv?


    return df


'0.23.1'

In [ ]:
# helper functions for text_preprocess
def remove_html_tags(text):
    no_html = re.sub('<.*?>','',text)
    return no_html

def remove_punctuation(text):
    punctuation_free = "".join([i for i in text if i not in string.punctuation])
    return punctuation_free

In [ ]:
'''
Options:
- html -- remove html headers and syntax
- stop -- remove stopwords
- punc -- remove punctuation
- lower -- change entire string to lowercase
'''
def text_preprocess(df, options):
    df["Clean_text"] = df["Text"]
    
    if 'html' in options:
        df["Clean_text"] = df["Clean_text"].apply(lambda x: remove_html_tags(x))

    if 'stop' in options:
        df["Clean_text"] = df["Clean_text"].apply(lambda x: remove_stopwords(x))

    if 'punc':
        df["Clean_text"] = df["Clean_text"].apply(lambda x: remove_punctuation(x))

    if 'lower':
        df["Clean_text"] = df["Clean_text"].apply(lambda x: x.lower())


In [ ]:
'''
X_train : should be a series, df['Text'] (according to EDA ipynb)
'''
def vectorize(X_train, y_train):
    # tokenizer
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train) 

    # tf-idf
    tfidf_transformer = TfidfTransformer(use_idf=False)
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


    return X_train_tfidf